# 2001 Longstaff Numerical Example

In [11]:
# Least-squares estimation of conditional expected payoff to value American options
# Adapted from: 
#    Longstaff, Francis & Schwartz, Eduardo. (2001). 
#    Valuing American Options by Simulation: A Simple Least-Squares Approach. 
#    Review of Financial Studies. 14. 113-47. 10.1093/rfs/14.1.113. 

# American Put Option on share of non-dividend-paying stock exercisable at times 1, 2, 3
# Sample paths generated under risk-neutral assumption

# riskless rate of 6%
r_r = 0.06
discount_factor = round.(exp(-(r_r)); digits=5)

# risk-neutral stock price paths
using DataFrames
Path = ["1","2","3","4","5","6","7","8"]
t_zero = [1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00] 
t_one = [1.09,1.16,1.22,0.93,1.11,0.76,0.92,0.88]
t_two = [1.08,1.26,1.07,0.97,1.56,0.77,0.84,1.22]
t_three = [1.34,1.54,1.03,0.92,1.52,0.90,1.01,1.34]
df = DataFrame(Path=Path,t_zero=t_zero,t_one=t_one,t_two=t_two,t_three=t_three)
println("Stock price paths")
println(df)
println("")

#optimal cash-flow conditional on not exercising option before final expiration date, time 3
df_time_three = df[!,:t_three]

cash_flow_t3 = Float64[]

str_price = 1.10

for i = view(df_time_three,:,1)
    if i >= str_price
        i = 0.0
    else
        i = str_price - i
    end
    push!(cash_flow_t3,round.(i; digits=2))
end

df_t3 = DataFrame(Path=Path,t_three=cash_flow_t3)
println("")
println("Cash-flow matrix at time 3")
println(df_t3)


Stock price paths
8×5 DataFrame
 Row │ Path    t_zero   t_one    t_two    t_three 
     │ String  Float64  Float64  Float64  Float64 
─────┼────────────────────────────────────────────
   1 │ 1           1.0     1.09     1.08     1.34
   2 │ 2           1.0     1.16     1.26     1.54
   3 │ 3           1.0     1.22     1.07     1.03
   4 │ 4           1.0     0.93     0.97     0.92
   5 │ 5           1.0     1.11     1.56     1.52
   6 │ 6           1.0     0.76     0.77     0.9
   7 │ 7           1.0     0.92     0.84     1.01
   8 │ 8           1.0     0.88     1.22     1.34


Cash-flow matrix at time 3
8×2 DataFrame
 Row │ Path    t_three 
     │ String  Float64 
─────┼─────────────────
   1 │ 1          0.0
   2 │ 2          0.0
   3 │ 3          0.07
   4 │ 4          0.18
   5 │ 5          0.0
   6 │ 6          0.2
   7 │ 7          0.09
   8 │ 8          0.0


In [3]:
# in-the-money paths for the option at time 2

df_time_two = df[!,:t_two]

cash_flow_t2 = Float64[]

for i = view(df_time_two,:,1)
    if i >= str_price
        i = NaN
    else
        i = i
    end
    push!(cash_flow_t2,round.(i; digits=2))
end

# discounted cash flows received at time 3
Y_t2 = cash_flow_t3 .* discount_factor

Regress_at_t2 = DataFrame(Path = Path, Y = Y_t2, X = cash_flow_t2)
filter!(row -> ! isnan(row.X), Regress_at_t2)
println("Regression at time 2 (discount factor included in Y)")
println(Regress_at_t2)


Regression at time 2 (discount factor included in Y)
5×3 DataFrame
 Row │ Path    Y          X       
     │ String  Float64    Float64 
─────┼────────────────────────────
   1 │ 1       0.0           1.08
   2 │ 3       0.0659232     1.07
   3 │ 4       0.169517      0.97
   4 │ 6       0.188352      0.77
   5 │ 7       0.0847584     0.84


In [5]:
#Regressing Y on basis functions 

Y_t2 = Regress_at_t2[:,2]
X = Regress_at_t2[:,3]

basis_X = ([X.^j for j in range(0, 2,step=1)])

basisfnx = hcat(basis_X...)

theta = (inv(basisfnx' * basisfnx)) * basisfnx' * Y_t2

using Printf

#conditional expectation function for time 2
@printf("The conditional expectation function E[Y|X] is: %.3f + %.3f X + %.3f X^2 \n"
        , theta[1], theta[2], theta[3])

#estimating cash flow from continuing option's life conditional on stock price at time 2
Continuation_value_t2 = basisfnx * theta

#value of immediate exercise at time 2
immediate_exercise_t2 = Float64[]

for i = view(df_time_two,:,1)
    if i >= str_price
        i = 0.0
    else
        i = str_price - i
    end
    push!(immediate_exercise_t2,round.(i; digits=2))
end

#Optimal exercise decision at time 2
forlater = copy(cash_flow_t3) #for future cash flow matrix at time 2
Optimal_t2 = DataFrame(Path = Path, Exercise = immediate_exercise_t2, X = cash_flow_t2, forlater = forlater)
filter!(row -> ! isnan(row.X), Optimal_t2)
cashflowt2matrix_t3 = copy(Optimal_t2[!,4])
cashflowt2matrix_Path = copy(Optimal_t2[!,1])

Optimal_t2[!,:Continuation] = round.(Continuation_value_t2; digits=4)
select!(Optimal_t2, Not(:X))
select!(Optimal_t2, Not(:forlater))
println("")
println("Optimal early exercise decision at time 2")
println(Optimal_t2)

The conditional expectation function E[Y|X] is: -1.070 + 2.983 X + -1.814 X^2 

Optimal early exercise decision at time 2
5×3 DataFrame
 Row │ Path    Exercise  Continuation 
     │ String  Float64   Float64      
─────┼────────────────────────────────
   1 │ 1           0.02        0.0367
   2 │ 3           0.03        0.0459
   3 │ 4           0.13        0.1175
   4 │ 6           0.33        0.152
   5 │ 7           0.26        0.1564


In [6]:
#optimal cash flows recieved by the optionholder conditional on not exercising prior to time 2

cashflowt2matrix_t2 = copy(Optimal_t2[!,2])

for i = 1:5
    if cashflowt2matrix_t2[i] < Continuation_value_t2[i]
        cashflowt2matrix_t2[i] = 0.0
    else
        cashflowt2matrix_t3[i] = 0.0
    end
end


println("Cash-flow matrix at time 2")
cashflowt2matrix = DataFrame(Path = cashflowt2matrix_Path,
                            t_two = cashflowt2matrix_t2, t_three = cashflowt2matrix_t3)

for i = 1:8
    if Path[i] ∉ cashflowt2matrix_Path
        foreach((v,n) -> insert!(cashflowt2matrix[!,n], i, v), [string(i), 0.0, 0.0], names(cashflowt2matrix))       
    end
end

println(cashflowt2matrix)


Cash-flow matrix at time 2
8×3 DataFrame
 Row │ Path    t_two    t_three 
     │ String  Float64  Float64 
─────┼──────────────────────────
   1 │ 1          0.0      0.0
   2 │ 2          0.0      0.0
   3 │ 3          0.0      0.07
   4 │ 4          0.13     0.0
   5 │ 5          0.0      0.0
   6 │ 6          0.33     0.0
   7 │ 7          0.26     0.0
   8 │ 8          0.0      0.0


In [7]:
#Regressing to determine if option should be exercised at time 1

df_time_one = df[!,:t_one]

Regresst1_Y = cashflowt2matrix[!, :t_two]
Regresst1_X = Float64[]

for i = view(df_time_one,:,1)
    if i >= str_price
        i = NaN
    else
        i = i
    end
    push!(Regresst1_X,round.(i; digits=2))
end

#discounting actual cash flows rather than conditional expected value to avoid upward bias in option value
Y_t1 = Regresst1_Y .* discount_factor

Regress_at_t1 = DataFrame(Path = Path, Y = Y_t1, X = Regresst1_X)
filter!(row -> ! isnan(row.X), Regress_at_t1)
println("Regression at time 1 (discount factor included in Y)")
println(Regress_at_t1)

Regression at time 1 (discount factor included in Y)
5×3 DataFrame
 Row │ Path    Y         X       
     │ String  Float64   Float64 
─────┼───────────────────────────
   1 │ 1       0.0          1.09
   2 │ 4       0.122429     0.93
   3 │ 6       0.310781     0.76
   4 │ 7       0.244858     0.92
   5 │ 8       0.0          0.88


In [8]:
#Regressing Y_t1 on basis functions

Y_t1 = Regress_at_t1[:,2]
X_t1 = Regress_at_t1[:,3]

basis_X_t1 = ([X_t1.^j for j in range(0, 2,step=1)])

basisfnx_t1 = hcat(basis_X_t1...)

theta_t1 = (inv(basisfnx_t1' * basisfnx_t1)) * basisfnx_t1' * Y_t1

using Printf

#conditional expectation function for time 1
@printf("The conditional expectation function E[Y|X] is: %.3f + %.3f X + %.3f X^2 \n", 
    theta_t1[1], theta_t1[2], theta_t1[3])

#estimating cash flow from continuing option's life conditional on stock price at time 1
Continuation_value_t1 = basisfnx_t1 * theta_t1

#value of immediate exercise at time 1
immediate_exercise_t1 = Float64[]

for i = view(df_time_one,:,1)
    if i >= str_price
        i = 0.0
    else
        i = str_price - i
    end
    push!(immediate_exercise_t1,round.(i; digits=2))
end

#Optimal exercise decision at time 1
forlater_t1 = cashflowt2matrix[:,2] #for future cash flow matrix at time 1
Optimal_t1 = DataFrame(Path = Path, Exercise = immediate_exercise_t1, X = Regresst1_X, forlater = forlater_t1)
filter!(row -> ! isnan(row.X), Optimal_t1)
cashflowt1matrix_t2 = copy(Optimal_t1[!,4])
cashflowt1matrix_Path = copy(Optimal_t1[!,1])

Optimal_t1[!,:Continuation] = round.(Continuation_value_t1; digits=4)
select!(Optimal_t1, Not(:X))
select!(Optimal_t1, Not(:forlater))
println("")
println("Optimal early exercise decision at time 1")
println(Optimal_t1)


The conditional expectation function E[Y|X] is: 2.038 + -3.335 X + 1.356 X^2 

Optimal early exercise decision at time 1
5×3 DataFrame
 Row │ Path    Exercise  Continuation 
     │ String  Float64   Float64      
─────┼────────────────────────────────
   1 │ 1           0.01        0.0135
   2 │ 4           0.17        0.1087
   3 │ 6           0.34        0.2861
   4 │ 7           0.18        0.117
   5 │ 8           0.22        0.1528


In [9]:
#optimal cash flows determined by following stopping rule

cashflowt1matrix_t1 = copy(Optimal_t1[!,2])

for i = 1:5
    if cashflowt1matrix_t1[i] < Continuation_value_t1[i]
        cashflowt1matrix_t1[i] = 0.0
    else
        cashflowt1matrix_t2[i] = 0.0
    end
end

println("Option cash-flow matrix")
cashflowt1matrix = DataFrame(Path = cashflowt1matrix_Path,
                            t_one = cashflowt1matrix_t1, t_two = cashflowt1matrix_t2)

for i = 1:8
    if Path[i] ∉ cashflowt1matrix_Path
        foreach((v,n) -> insert!(cashflowt1matrix[!,n], i, v), [string(i), 0.0, 0.0], names(cashflowt1matrix))       
    end
end

cashflowt2matrix[:,3]
insertcols!(cashflowt1matrix, 4, :t_three => cashflowt2matrix[:,3])
println(cashflowt1matrix)


Option cash-flow matrix
8×4 DataFrame
 Row │ Path    t_one    t_two    t_three 
     │ String  Float64  Float64  Float64 
─────┼───────────────────────────────────
   1 │ 1          0.0       0.0     0.0
   2 │ 2          0.0       0.0     0.0
   3 │ 3          0.0       0.0     0.07
   4 │ 4          0.17      0.0     0.0
   5 │ 5          0.0       0.0     0.0
   6 │ 6          0.34      0.0     0.0
   7 │ 7          0.18      0.0     0.0
   8 │ 8          0.22      0.0     0.0


In [10]:
# Option Valuation for American Put Option
discountt1_Y = cashflowt1matrix[!,:t_one] .* discount_factor
discountt2_Y = cashflowt1matrix[!,:t_two] .* (discount_factor)^2
discountt3_Y = cashflowt1matrix[!,:t_three] .* (discount_factor)^3

discount_val = DataFrame(discountt1_Y = discountt1_Y, discountt2_Y = discountt2_Y, discountt3_Y = discountt3_Y)
AmericanOption_value = sum(eachrow(sum(eachcol(discount_val)))) / length(discount_val[:,1])

using Printf

@printf("The value of the American Put Option is: %.4f \n", AmericanOption_value[1])

# Option Valuation for European Put Option
discount_euro = cash_flow_t3 .* (discount_factor)^3
EuroOption_value = sum(eachrow(discount_euro)) / length(cash_flow_t3)

using Printf

@printf("The value of the European Put Option is: %.4f \n", EuroOption_value[1])


The value of the American Put Option is: 0.1144 
The value of the European Put Option is: 0.0564 
